In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [8]:
path = '../res/tristan_md_trajectory_N125_F500_L10.0.xyz'

with open(path) as f:
    
    natoms = int(f.readline()) # per configurations
    nconfig = 1

    comment = f.readline().split()
    box_size = np.array(comment[9:12]).astype(float)
    frame = [comment[1]]
    time = [float(comment[4])]

    elements = []
    atompos = []
    configpos = []

    for line in f:

        split = line.split()
        nsplit = len(split)
        if nsplit > 4:

            frame.append(split[1])
            time.append(float(split[4]))
            atompos.append(configpos)
            configpos = []
            nconfig += 1

        elif nsplit == 4:

            if nconfig == 1:

                elements.append(split[0])

            configpos.append(split[1:4])

    atompos.append(configpos)
    atompos = np.array(atompos).astype(float)
    frame = np.array(frame)
    time = np.array(time)

In [19]:
from numba import njit, prange

@njit(parallel=True)
def msd(tau:float):

    assert 0 <= tau <= time[-1], f'Time interval must be between 0 and {time[-1]}'

    m = int(np.floor(tau / (time[1]-time[0])))
    s = 0

    for i in prange(natoms):

        for k in range(time.size - m):

            s += np.sum((atompos[k+m, i] - atompos[k, i])**2)

    return s/(time.size - m)/natoms